In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U -q kaggle

In [3]:
!mkdir -p ~/.kaggle

In [4]:
from google.colab import files

In [5]:
files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"miguelariaslondoo","key":"7418f5016cd15765af60765f150c1ba7"}'}

In [6]:
!cp kaggle.json ~/.kaggle/

In [7]:
!kaggle competitions download -c champs-scalar-coupling

100% 377M/377M [00:04<00:00, 56.2MB/s]
100% 377M/377M [00:04<00:00, 97.6MB/s]


In [ ]:
!unzip -o '/content/champs-scalar-coupling.zip'

In [2]:
import glob
import pandas as pd

csv_files = glob.glob('*.csv')

for file in csv_files:
    df = pd.read_csv(file)
    print(f"Information for {file}:")
    print(f"Number of rows: {len(df)}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"Data types:\n{df.dtypes}\n")


Information for structures.csv:
Number of rows: 2358875
Columns: ['molecule_name', 'atom_index', 'atom', 'x', 'y', 'z']
Data types:
molecule_name     object
atom_index         int64
atom              object
x                float64
y                float64
z                float64
dtype: object

Information for magnetic_shielding_tensors.csv:
Number of rows: 2358875
Columns: ['molecule_name', 'atom_index', 'XX', 'YX', 'ZX', 'XY', 'YY', 'ZY', 'XZ', 'YZ', 'ZZ']
Data types:
molecule_name     object
atom_index         int64
XX               float64
YX               float64
ZX               float64
XY               float64
YY               float64
ZY               float64
XZ               float64
YZ               float64
ZZ               float64
dtype: object

Information for df_final_train.csv:
Number of rows: 4659076
Columns: ['molecule_name', 'atom_index_0', 'atom_index_1', 'dipole_moment_x', 'dipole_moment_y', 'dipole_moment_z', 'potential_energy', 'type', 'scalar_coupling_constant', 'f

In [3]:
for file in csv_files:
    df = pd.read_csv(file)
    print(f"{file}:{df.shape}")

structures.csv:(2358875, 6)
magnetic_shielding_tensors.csv:(2358875, 11)
df_final_train.csv:(4659076, 41)
dipole_moments.csv:(130789, 4)
sample_submission.csv:(2505190, 2)
mulliken_charges.csv:(2358875, 3)
scalar_coupling_contributions.csv:(4659076, 8)
potential_energy.csv:(130789, 2)
train.csv:(4659076, 6)
test.csv:(2505190, 5)


CARGAMOS TODOS LOS DATOS EN SUS RESPECTIVOS DATAFRAMES


In [4]:
import pandas as pd
import matplotlib.pyplot as plt

df_dipole_moments = pd.read_csv('dipole_moments.csv')
df_magnetic_shielding_tensors = pd.read_csv('magnetic_shielding_tensors.csv')
df_mulliken_charges = pd.read_csv('mulliken_charges.csv')
df_potential_energy = pd.read_csv('potential_energy.csv')
#df_sample_submission = pd.read_csv('sample_submission.csv')
df_scalar_coupling_contributions = pd.read_csv('scalar_coupling_contributions.csv')
df_structures = pd.read_csv('structures.csv')
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')


## ORGANIZAMOS LOS DATOS ADICIONALES PARA AGREGARLOS AL DATAFRAME DE TRAIN Y TEST

In [5]:
# Data frame of dipole moments and potential energy
# have the same size,are he total number of molecules by name
df_dipole_moments_potential_energy = pd.merge(df_dipole_moments, df_potential_energy, on='molecule_name')
#rename the columns x y z to dipole_moment_x dipole_moment_y dipole_moment_z
df_dipole_moments_potential_energy.rename(columns={'X': 'dipole_moment_x', 'Y': 'dipole_moment_y', 'Z': 'dipole_moment_z'}, inplace=True)
print(df_dipole_moments_potential_energy.head())
print(df_dipole_moments_potential_energy.shape)

      molecule_name  dipole_moment_x  dipole_moment_y  dipole_moment_z  \
0  dsgdb9nsd_000001           0.0000              0.0           0.0000   
1  dsgdb9nsd_000002          -0.0002              0.0           1.6256   
2  dsgdb9nsd_000003           0.0000              0.0          -1.8511   
3  dsgdb9nsd_000004           0.0000              0.0           0.0000   
4  dsgdb9nsd_000005           0.0000              0.0          -2.8937   

   potential_energy  
0        -40.523680  
1        -56.560246  
2        -76.426077  
3        -77.335268  
4        -93.428488  
(130789, 5)


In [6]:
#data frame of magnetic shielding tensors,structures and mulliken charges
print(f"shape of magshield {df_magnetic_shielding_tensors.shape}")
print(f"shape of mulliken {df_mulliken_charges.shape}")
print(f"shape of structures {df_structures.shape}")
#merge the data frames y molecule name and atom index
df_magnetic_shielding_tensors_mulliken_charges_structures = pd.merge(df_magnetic_shielding_tensors, df_mulliken_charges, on=['molecule_name','atom_index'])
df_magnetic_shielding_tensors_mulliken_charges_structures = pd.merge(df_magnetic_shielding_tensors_mulliken_charges_structures, df_structures, on=['molecule_name','atom_index'])
print(df_magnetic_shielding_tensors_mulliken_charges_structures.head())

shape of magshield (2358875, 11)
shape of mulliken (2358875, 3)
shape of structures (2358875, 6)
      molecule_name  atom_index        XX      YX      ZX      XY        YY  \
0  dsgdb9nsd_000001           0  195.3147  0.0000 -0.0001  0.0000  195.3171   
1  dsgdb9nsd_000001           1   31.3410 -1.2317  4.0544 -1.2317   28.9546   
2  dsgdb9nsd_000001           2   31.5814  1.2173 -4.1474  1.2173   28.9036   
3  dsgdb9nsd_000001           3   31.5172  4.1086  1.2723  4.1088   33.9068   
4  dsgdb9nsd_000001           4   31.4029 -4.0942 -1.1793 -4.0944   34.0776   

       ZY      XZ      YZ        ZZ  mulliken_charge atom         x         y  \
0  0.0007 -0.0001  0.0007  195.3169        -0.535689    C -0.012698  1.085804   
1 -1.7173  4.0546 -1.7173   34.0861         0.133921    H  0.002150 -0.006031   
2 -1.6036 -4.1476 -1.6036   33.8967         0.133922    H  1.011731  1.463751   
3  1.6950  1.2724  1.6951   28.9579         0.133923    H -0.540815  1.447527   
4  1.6259 -1.1795  1.62

In [7]:
#data frame of train and scalar coupling contributions,have the same size
print(f"shape of train {df_train.shape}")
print(f"shape of scalar coupling {df_scalar_coupling_contributions.shape}")
# merging the 2 data frames by molecule name,type and atom index
df_train_scalar_coupling_contributions = pd.merge(df_train, df_scalar_coupling_contributions, on=['molecule_name','atom_index_0','atom_index_1','type'])
print(df_train_scalar_coupling_contributions.head())

shape of train (4659076, 6)
shape of scalar coupling (4659076, 8)
   id     molecule_name  atom_index_0  atom_index_1  type  \
0   0  dsgdb9nsd_000001             1             0  1JHC   
1   1  dsgdb9nsd_000001             1             2  2JHH   
2   2  dsgdb9nsd_000001             1             3  2JHH   
3   3  dsgdb9nsd_000001             1             4  2JHH   
4   4  dsgdb9nsd_000001             2             0  1JHC   

   scalar_coupling_constant       fc        sd      pso       dso  
0                   84.8076  83.0224  0.254579  1.25862  0.272010  
1                  -11.2570 -11.0347  0.352978  2.85839 -3.433600  
2                  -11.2548 -11.0325  0.352944  2.85852 -3.433870  
3                  -11.2543 -11.0319  0.352934  2.85855 -3.433930  
4                   84.8074  83.0222  0.254585  1.25861  0.272013  


In [8]:
#Now we have to merge df_dipole_moments_potential_energy and df_train_scalar_coupling_contributions
#by molecule name,the is only one molecule name in df_dipole_moments_potential_energy so that means that
#so it must aply to all the rows of df_train_scalar_coupling_contributions by the same name
df_dipole_moments_potential_energy_train_scalar_coupling_contributions = pd.merge(df_dipole_moments_potential_energy, df_train_scalar_coupling_contributions, on=['molecule_name'])

#errase de id column
df_dipole_moments_potential_energy_train_scalar_coupling_contributions.drop(['id'], axis=1, inplace=True)
# putting atom_index_0 and atom_index_1 next to molecule_name
cols = list(df_dipole_moments_potential_energy_train_scalar_coupling_contributions.columns.values)
cols.pop(cols.index('molecule_name'))
cols.pop(cols.index('atom_index_0'))
cols.pop(cols.index('atom_index_1'))
df_dipole_moments_potential_energy_train_scalar_coupling_contributions = df_dipole_moments_potential_energy_train_scalar_coupling_contributions[['molecule_name']+['atom_index_0']+['atom_index_1']+cols]
print(df_dipole_moments_potential_energy_train_scalar_coupling_contributions.head())

      molecule_name  atom_index_0  atom_index_1  dipole_moment_x  \
0  dsgdb9nsd_000001             1             0              0.0   
1  dsgdb9nsd_000001             1             2              0.0   
2  dsgdb9nsd_000001             1             3              0.0   
3  dsgdb9nsd_000001             1             4              0.0   
4  dsgdb9nsd_000001             2             0              0.0   

   dipole_moment_y  dipole_moment_z  potential_energy  type  \
0              0.0              0.0         -40.52368  1JHC   
1              0.0              0.0         -40.52368  2JHH   
2              0.0              0.0         -40.52368  2JHH   
3              0.0              0.0         -40.52368  2JHH   
4              0.0              0.0         -40.52368  1JHC   

   scalar_coupling_constant       fc        sd      pso       dso  
0                   84.8076  83.0224  0.254579  1.25862  0.272010  
1                  -11.2570 -11.0347  0.352978  2.85839 -3.433600  
2       

Now we have to merge df_dipole_moments_potential_energy_train_scalar_coupling_contributions

with df_magnetic_shielding_tensors_mulliken_charges_structures,by molecule name,atom_index_0 and atom_index_1
the problem is that df_dipole_moments_potential_energy_train_scalar_coupling_contributions has 4658147 rows
and df_magnetic_shielding_tensors_mulliken_charges_structures has 1533537 rows
so we have to do a merge by molecule name,atom_index_0 and atom_index_1
but we have to do it 2 times,one for each atom_index_0 and atom_index_1
first we do it for atom_index_0,but we need to create 2 dataframes of df_magnetic_shielding_tensors_mulliken_charges_structures
one for atom_index_0 and another for atom_index_1

In [9]:

# first we create df_magnetic_shielding_tensors_mulliken_charges_structures_atom_index_0
df_magnetic_shielding_tensors_mulliken_charges_structures_atom_index_0 = df_magnetic_shielding_tensors_mulliken_charges_structures.copy()
# rename all the columns of df_magnetic_shielding_tensors_mulliken_charges_structures_atom_index_0 adig a AT0 to the end of each column name
# except atom_index and molecule_name
df_magnetic_shielding_tensors_mulliken_charges_structures_atom_index_0.rename(columns=lambda x: x + '_AT0' if (x != 'atom_index' and x != 'molecule_name') else x, inplace=True)
df_magnetic_shielding_tensors_mulliken_charges_structures_atom_index_0.rename(columns={'atom_index': 'atom_index_0'}, inplace=True)
print(df_magnetic_shielding_tensors_mulliken_charges_structures_atom_index_0.head())
#we do the same for df_magnetic_shielding_tensors_mulliken_charges_structures_atom_index_1
df_magnetic_shielding_tensors_mulliken_charges_structures_atom_index_1 = df_magnetic_shielding_tensors_mulliken_charges_structures.copy()
df_magnetic_shielding_tensors_mulliken_charges_structures_atom_index_1.rename(columns=lambda x: x + '_AT1' if (x != 'atom_index' and x != 'molecule_name') else x, inplace=True)
df_magnetic_shielding_tensors_mulliken_charges_structures_atom_index_1.rename(columns={'atom_index': 'atom_index_1'}, inplace=True)
print(df_magnetic_shielding_tensors_mulliken_charges_structures_atom_index_1.head())



      molecule_name  atom_index_0    XX_AT0  YX_AT0  ZX_AT0  XY_AT0    YY_AT0  \
0  dsgdb9nsd_000001             0  195.3147  0.0000 -0.0001  0.0000  195.3171   
1  dsgdb9nsd_000001             1   31.3410 -1.2317  4.0544 -1.2317   28.9546   
2  dsgdb9nsd_000001             2   31.5814  1.2173 -4.1474  1.2173   28.9036   
3  dsgdb9nsd_000001             3   31.5172  4.1086  1.2723  4.1088   33.9068   
4  dsgdb9nsd_000001             4   31.4029 -4.0942 -1.1793 -4.0944   34.0776   

   ZY_AT0  XZ_AT0  YZ_AT0    ZZ_AT0  mulliken_charge_AT0 atom_AT0     x_AT0  \
0  0.0007 -0.0001  0.0007  195.3169            -0.535689        C -0.012698   
1 -1.7173  4.0546 -1.7173   34.0861             0.133921        H  0.002150   
2 -1.6036 -4.1476 -1.6036   33.8967             0.133922        H  1.011731   
3  1.6950  1.2724  1.6951   28.9579             0.133923        H -0.540815   
4  1.6259 -1.1795  1.6260   28.9013             0.133923        H -0.523814   

      y_AT0     z_AT0  
0  1.085804  0

In [10]:
#the names are very long so we rename the dataframe
df_pm_pe_t_scc = df_dipole_moments_potential_energy_train_scalar_coupling_contributions.copy()
#now we merge the first time df_dipole_moments_potential_energy_train_scalar_coupling_contributions
# with df_magnetic_shielding_tensors_mulliken_charges_structures_atom_index_0
df_pm_pe_t_scc_mst_mcs_ai0 = pd.merge(df_pm_pe_t_scc, df_magnetic_shielding_tensors_mulliken_charges_structures_atom_index_0, on=['molecule_name','atom_index_0'])
#now we merge the second time df_dipole_moments_potential_energy_train_scalar_coupling_contributions
# with df_magnetic_shielding_tensors_mulliken_charges_structures_atom_index_1
df_pm_pe_t_scc_mst_mcs_ai0_ai1 = pd.merge(df_pm_pe_t_scc_mst_mcs_ai0, df_magnetic_shielding_tensors_mulliken_charges_structures_atom_index_1, on=['molecule_name','atom_index_1'])
print(df_pm_pe_t_scc_mst_mcs_ai0_ai1.head())

      molecule_name  atom_index_0  atom_index_1  dipole_moment_x  \
0  dsgdb9nsd_000001             1             0              0.0   
1  dsgdb9nsd_000001             2             0              0.0   
2  dsgdb9nsd_000001             3             0              0.0   
3  dsgdb9nsd_000001             4             0              0.0   
4  dsgdb9nsd_000001             1             2              0.0   

   dipole_moment_y  dipole_moment_z  potential_energy  type  \
0              0.0              0.0         -40.52368  1JHC   
1              0.0              0.0         -40.52368  1JHC   
2              0.0              0.0         -40.52368  1JHC   
3              0.0              0.0         -40.52368  1JHC   
4              0.0              0.0         -40.52368  2JHH   

   scalar_coupling_constant       fc  ...    YY_AT1  ZY_AT1  XZ_AT1  YZ_AT1  \
0                   84.8076  83.0224  ...  195.3171  0.0007 -0.0001  0.0007   
1                   84.8074  83.0222  ...  195.3171  0

In [11]:
df_final_train=df_pm_pe_t_scc_mst_mcs_ai0_ai1.copy()
print(f"Columns: {df_final_train.columns.tolist()}")
print({len(df_final_train.columns.tolist())})
print(df_final_train.shape)


Columns: ['molecule_name', 'atom_index_0', 'atom_index_1', 'dipole_moment_x', 'dipole_moment_y', 'dipole_moment_z', 'potential_energy', 'type', 'scalar_coupling_constant', 'fc', 'sd', 'pso', 'dso', 'XX_AT0', 'YX_AT0', 'ZX_AT0', 'XY_AT0', 'YY_AT0', 'ZY_AT0', 'XZ_AT0', 'YZ_AT0', 'ZZ_AT0', 'mulliken_charge_AT0', 'atom_AT0', 'x_AT0', 'y_AT0', 'z_AT0', 'XX_AT1', 'YX_AT1', 'ZX_AT1', 'XY_AT1', 'YY_AT1', 'ZY_AT1', 'XZ_AT1', 'YZ_AT1', 'ZZ_AT1', 'mulliken_charge_AT1', 'atom_AT1', 'x_AT1', 'y_AT1', 'z_AT1']
{41}
(4659076, 41)


In [15]:
#Saving the DF to a .csv file
df_final_train.to_csv('df_final_train.csv', index=False)

In [12]:
# porcentage of value types in the data frame
print((df_final_train.dtypes.value_counts(normalize=True) * 100).round(0).astype(str) + '%')

float64    85.0%
object     10.0%
int64       5.0%
dtype: object


In [13]:
print("df_train_merged_final.csv:")
print(f"Number of rows: {len(df_final_train)}")
print(f"Number of Columns: {len((df_final_train.columns.tolist()))}")
print(f"Data types:\n{df_final_train.dtypes}\n")

df_train_merged_final.csv:
Number of rows: 4659076
Number of Columns: 41
Data types:
molecule_name                object
atom_index_0                  int64
atom_index_1                  int64
dipole_moment_x             float64
dipole_moment_y             float64
dipole_moment_z             float64
potential_energy            float64
type                         object
scalar_coupling_constant    float64
fc                          float64
sd                          float64
pso                         float64
dso                         float64
XX_AT0                      float64
YX_AT0                      float64
ZX_AT0                      float64
XY_AT0                      float64
YY_AT0                      float64
ZY_AT0                      float64
XZ_AT0                      float64
YZ_AT0                      float64
ZZ_AT0                      float64
mulliken_charge_AT0         float64
atom_AT0                     object
x_AT0                       float64
y_AT0          

In [4]:
df_final_train = pd.read_csv('df_final_train.csv')

In [7]:
df_final_train_test = df_final_train.copy()

In [8]:
import pandas as pd
import numpy as np

# Select 6 random columns
cols = df_final_train_test.sample(n=6, axis=1).columns

# Replace 5% of the data in those columns with NaN
for col in cols:
    df_final_train_test.loc[df_final_train_test.sample(frac=0.05).index, col] = np.nan


In [9]:
# Calculate the percentage of missing values for each column
missing_percentages = (df_final_train_test.isnull().sum() / len(df_final_train_test) * 100).round(0).astype(str) + '%'
# Printing only the colums that have missing values
print(missing_percentages[missing_percentages != '0.0%'])

dipole_moment_z             5.0%
scalar_coupling_constant    5.0%
fc                          5.0%
sd                          5.0%
ZZ_AT0                      5.0%
ZZ_AT1                      5.0%
dtype: object
